# Binance future tickers to onchain mappings

In this notebook, we show the corresponding onchain asset details for Binance futures. Not all Binance futures are supported on chain, but the ones that are are listed below.

In [72]:
# https://api.binance.com/api/v3/exchangeInfo

from tradingstrategy.binance.downloader import BinanceDownloader

downloader = BinanceDownloader()
margin_assets = list(downloader.fetch_assets('MARGIN'))
print(len(margin_assets))

597


## Future mapping format

Every key in the dictionary has one or more items that adhere to the following format: 

Key: Binance ticker (see the `margin_assets` variable created above)
Value: Item or list of items where each item has 7 values: 

    (chain_id, exchange, address, tvl, swap fee, reserve_currency, has_lending, total_supplied)

- In some cases, both assets are volatile, instead of the typical case where one is volatile and the other is stable. In these cases, the `reserve_currency` just set to a placeholder value.
- The `tvl` and `total_supplied` values were recorded as at 14/15-05-2024. Both are in millions of USD. 

### Lending information

- Aave does not exist on binance smart chain and we don't have data for Avalanche, so we only check ethereum, polygon and arbitrum
- `has_lending is True` if both assets in the trading pair are listed on Aave. In most of the cases, at least on the tokens (typically a stable coin) is listed on Aave, but not always both tokens.
- `total_supplied` is the total amount of the asset supplied to Aave for the `base_token` when the `quote_token` is a stable coin. When both assets are volatile, which is only relevant if `has_lending = True`, only happens when ETH is one of the tokens. The non-ETH token liquidity is given. E.g. for `ETHBTC`, the `total_supplied` is the total amount of BTC supplied to Aave. So, basically, we assume the that stable coins and ETH always have decent liquidity and so don't provide figures for these. Also, we prefer Aave V3 over V2.
- USDC, USDT, DAI are on Aave except TUSD, FDUSD, BUSD

In [73]:
from tradingstrategy.chain import ChainId
from tradeexecutor.strategy.strategy_module import ReserveCurrency 

binance_futures_mappings = {
    "ETHBTC": [
        (ChainId.ethereum, 'sushi', '0xceff51756c56ceffca006cd410b03ffc46dd3a58', 33.7, '0.3%', 'BTC', True, 2_030),
        (ChainId.ethereum, 'uniswap-v2', '0xbb2b8038a1640196fbe3e38816f3e67cba72d940', 11.87, "0.3%", 'BTC', True, 2_030),
        (ChainId.polygon, 'quickswap', '0xdc9232e2df177d7a12fdff6ecbab114e2231198d', 2.95, "0.3%", 'BTC', True, 110.59),
        (ChainId.binance, 'pancakeswap-v2', '0xd171b26e4484402de70e3ea256be5a2630d7e88d', 2.41, "0.25%", 'BTC', False),
        (ChainId.arbitrum, 'sparta-dex', "0xdd90a3da9688ccca100dd4da1786677bc459f069", 1.22, "0.3%", 'BTC', True, 226.34),
        (ChainId.arbitrum, 'sushi', '0x515e252b2b5c22b4b2b6df66c2ebeea871aa4d69', 0.431, '0.3%', 'BTC', True, 226.34),
    ],
    "BNBBTC": (ChainId.binance, 'pancakeswap-v2', '0x61eb789d75a95caa3ff50ed7e47b96c132fec082', 3.14, '0.25%', False),
    "BNBETH": (ChainId.binance, 'pancakeswap-v2', '0x74e4716e431f45807dcf19f284c7aa99f18a4fbc', 3.95, '0.25%', False),
    "BTCUSDT": [
        (ChainId.ethereum, "uniswap-v3", '0x9db9e0e53058c89e5b94e29621a205198648425b', 9.71, "0.3%", ReserveCurrency.usdt, True, 2_030),
        (ChainId.ethereum, "uniswap-v3", '0x99ac8ca7087fa4a2a1fb6357269965a2014abc35', 20.0, "0.3%", ReserveCurrency.usdc, True, 2_030),
        (ChainId.ethereum, "uniswap-v3", '0x9a772018fbd77fcd2d25657e5c547baff3fd7d16', 5.85, "0.05%", ReserveCurrency.usdc, True, 2_030),
        (ChainId.arbitrum, "uniswap-v3", '0x5969efdde3cf5c0d9a88ae51e47d721096a97203', 2.2, "0.05%", ReserveCurrency.usdt, True, 226.34),
        (ChainId.binance, "pancakeswap-v2", '0x5840b79d19c053a79923df738a251426d8879308', 1.36, "0.25%", ReserveCurrency.usdt, False),
        (ChainId.arbitrum, "uniswap-v3", '0xac70bd92f89e6739b3a08db9b6081a923912f73d', 0.26226, "0.05%", ReserveCurrency.usdc, True, 226.34),
        (ChainId.polygon, "quickswap", "0xf6a637525402643b0654a54bead2cb9a83c8b498", 0.282, "0.3%", ReserveCurrency.usdc, True, 110.38),
    ],
    "ETHUSDT": [
        (ChainId.ethereum, "uniswap-v3", '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640', 151.99, "0.05%", ReserveCurrency.usdc, True),
        (ChainId.arbitrum, "uniswap-v3", '0x641c00a822e8b671738d32a431a4fb6074e5c79d', 12.38, "0.05%", ReserveCurrency.usdt, True),
        (ChainId.arbitrum, "uniswap-v3", '0xc31e54c7a869b9fcbecc14363cf510d1c41fa443', 9.26, "0.05%", ReserveCurrency.usdc, True),
        (ChainId.binance, "biswap", '0x63b30de1a998e9e64fd58a21f68d323b9bcd8f85', 3.18, "0.3%", ReserveCurrency.usdt, False),
    ],
    "LINKETH": (ChainId.ethereum, 'uniswap-v2', "0xa2107fa5b38d9bbd2c461d6edf11b11a50f6b974", 2.63, '0.3%', 'ETH', True, 146.27),
    "XRTETH": (ChainId.ethereum, 'uniswap-v2', "0x3185626c14acb9531d19560decb9d3e5e80681b1", 0.846, '0.3%', 'ETH', False),
    "BNBUSDT": [
        (ChainId.binance, "pancakeswap-v2", "0x16b9a82891338f9ba80e2d6970fdda79d1eb0dae", 14.0, "0.25%", ReserveCurrency.usdt, False),
        (ChainId.binance, "pancakeswap-v2", "0xd99c7f6c65857ac913a8f880a4cb84032ab2fc5b", 0.7278, "0.25%", ReserveCurrency.usdc, False),
    ],
    "LTCUSDT": (ChainId.binance, "biswap", "0x412b607f4cbe9cae77c6f720a701cd60fa0ebd3f", 0.39692, "0.3%", ReserveCurrency.usdt, False),
    "TUSDUSDT": [
        (ChainId.arbitrum, "uniswap-v3", "0xad397a0472503b066ab4b311d66fa1f659f4cb61", 0.19242, "0.05%", ReserveCurrency.usdc, False),
        (ChainId.ethereum, "uniswap-v3", "0x69ef438a28e6d123f809c9be785c79745ab5a0b8", 0.144, "1%", ReserveCurrency.usdt, False),
    ],
    "TRXUSDT": [
        (ChainId.ethereum, "uniswap-v2", "0x9a84a1852bc7fb608794960960adb04666a12b41", 0.399, "0.3%", ReserveCurrency.usdc, False),
        (ChainId.binance, "pancakeswap-v2", "0x1f7df58c60a56bc8322d3e42d7d37a0383d42746", 0.143, "0.25%", ReserveCurrency.usdt, False),
    ],
    "USDCUSDT": (ChainId.arbitrum, "uniswap-v3", "0xbe3ad6a5669dc0b8b12febc03608860c31e2eef6", 2.22, "0.01%", ReserveCurrency.usdt, True),
    "LINKUSDT": [
        (ChainId.ethereum, "uniswap-v3", "0xfad57d2039c21811c8f2b5d5b65308aa99d31559", 0.588, "0.3%", ReserveCurrency.usdc, True, 146.27),
        (ChainId.ethereum, "uniswap-v3", "0xac5a2c404ebba22a869998089ac7893ff4e1f0a7", 0.1, "0.3%", ReserveCurrency.usdt, True, 146.27),
    ],
    "DOGEUSDT": (ChainId.ethereum, "uniswap-v2", "0xfcd13ea0b906f2f87229650b8d93a51b2e839ebd", 3.38, "0.3%", ReserveCurrency.usdt, False),
    "DUSKUSDT": (ChainId.ethereum, "uniswap-v3", "0xff29d3e552155180809ea3a877408a4620058086", 0.789, "0.3%", ReserveCurrency.usdt, False),
    "WANUSDT": (ChainId.ethereum, "sushi", "0x18c12a654f982f68a91e8e6a4170ca1fcb9d60a2", 0.126, "0.3%", ReserveCurrency.usdt, False),
    "FUNUSDT": (ChainId.ethereum, "uniswap-v3", "0x1a349a3397a8431eed8d94a05f88f9001117fcaa", 1.65, "0.01%", ReserveCurrency.usdt, False),
    "NKNUSDT": (ChainId.ethereum, "uniswap-v3", "0xf743ace0dd18b26ef95988486315dd7a39b07035", 0.116, "0.3%", ReserveCurrency.usdt, False),
    "IOTXUSDT": (ChainId.ethereum, "sushi", "0x868d966d6d3d94b115e8b003c443c7a7b04a3359", 0.1, "0.3%", ReserveCurrency.usdt, False),
    "DATAUSDT": [
        (ChainId.binance, "pancakeswap-v2", "0xe0648ec82eb04b565476b9dbc5208f180fda44a6", 0.14, "0.25%", ReserveCurrency.usdt, False),
        (ChainId.polygon, "quickswap", "0x44eb80e54a22c740c919607599a3bc46750a3b71", 0.212, "0.3%", ReserveCurrency.usdc, False),
    ],
    "SOLUSDT": (ChainId.binance, "babyswap", "0xae08c9d357731fd8d25681de753551be14c00405", 0.17, "0.3%", ReserveCurrency.usdt, False),
    "HIVEUSDT": (ChainId.binance, "pancakeswap-v2", "0xd8eeaabc6b99df2210cd568eecd60a77e1f3fb09", 0.866, "0.25%", ReserveCurrency.usdt, False),
    "SCUSDT": (ChainId.binance, "unknown", "0x1daca20a8b5904e511c4474cdfdece9fffbdbe4d", 0.162, "0.3%", ReserveCurrency.usdt, False),
    "SNXUSDT": (ChainId.ethereum, "uniswap-v3", "0x020c349a0541d76c16f501abc6b2e9c98adae892", 0.102, "1%", ReserveCurrency.usdc, True, 8.02),
    "BLZUSDT": (ChainId.ethereum, "uniswap-v2", "0x5fa9569b0ed6aa01e234468e6a15b77988b950df", 0.951, "0.3%", ReserveCurrency.usdc, False),
    "WBTCDAI": (ChainId.ethereum, "uniswap-v3", "0x391e8501b626c623d39474afca6f9e46c2686649", 0.098, "0.3%", ReserveCurrency.dai, True, 2_030),
    "ETHDAI": [
        (ChainId.ethereum, "uniswap-v3", "0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8", 25.0, "0.3%", ReserveCurrency.dai, True),
        (ChainId.ethereum, "uniswap-v3", "0x60594a405d53811d3bc4766596efd80fd545a270", 8.56, "0.05%", ReserveCurrency.dai, True),
        (ChainId.polygon, "sushi", "0x6ff62bfb8c12109e8000935a6de54dad83a4f39f", 0.418, "0.3%", ReserveCurrency.dai, True),
        (ChainId.arbitrum, "uniswap-v3", "0xa961f0473da4864c5ed28e00fcc53a3aab056c1b", 0.264, "0.3%", ReserveCurrency.dai, True),
    ],
    "USDTDAI": [
        (ChainId.ethereum, "uniswap-v3", "0x5777d92f208679db4b9778590fa3cab3ac9e2168", 73.0, "0.01%", ReserveCurrency.usdc, True),
        (ChainId.arbitrum, "uniswap-v3", "0x7f580f8a02b759c350e6b8340e7c2d4b8162b6a9", 5.88, "0.01%", ReserveCurrency.usdt, True),
        (ChainId.binance, "pancakeswap-v2", "0xb3c4217ab2b265bf8c69718d280e3708b5e50577", 1.06, "0.25%", ReserveCurrency.usdt, False),
        (ChainId.polygon, "quickswap", "0xf04adbf75cdfc5ed26eea4bbbb991db002036bdd", 0.313, "0.3%", ReserveCurrency.usdc, True),
        (ChainId.polygon, "uniswap-v3", "0xf369277650ad6654f25412ea8bfbd5942733babc", 0.282, "0.01%", ReserveCurrency.usdc, True),
    ],
    "LUNAUSDT": (ChainId.ethereum, "uniswap-v3", "0x066375dc02ec5b1447b801ab5f5e9599dcb2a45e", 19.53, "0.3%", ReserveCurrency.usdc, False),
    "UNIUSDT": [
        (ChainId.binance, "pancakeswpa-v2", "0xa56d15585a42adfa5ec3765c6a9434e5b06c5c4b", 0.742, "0.25%", ReserveCurrency.usdt, False),
        (ChainId.ethereum, "uniswap-v3", "0xd0fc8ba7e267f2bc56044a7715a489d851dc6d78", 0.356, "0.3%", ReserveCurrency.usdc, True, 9.61),
    ],
    "SUNUSDT": [
        (ChainId.arbitrum, "unknown", "0x0cff1ab38fedee38e0d9b925d9332effe4c53e17", 2.17, "0.3%", ReserveCurrency.usdt, False),
        (ChainId.binance, "pancakeswap-v2", "0x4c7cc204b04f5f3f1891db3f4af89895aec53c26", 0.708, "0.25%", ReserveCurrency.usdt, False),
    ],
    "AVAXBTC": (ChainId.avalanche, "trader-joe", "0xd5a37dc5c9a396a03dd1136fc76a1a02b1c88ffa", 0.425, "0.3%", 'BTC', False),
    "AVAXUSDT": [
        (ChainId.avalanche, "pangolin", "0x0e0100ab771e9288e0aa97e11557e6654c3a9665", 2.62, "0.3%", ReserveCurrency.usdc, False),
        (ChainId.avalanche, "trader-joe", "0xed8cbd9f0ce3c6986b22002f03c6475ceb7a6256", 0.981, "0.3%", ReserveCurrency.usdt, False),
    ],
    "ORNUSDT": (ChainId.binance, "minereum-2", "0x684fd3c5cdfe279c82adc33029b095292d31ca35", 0.363, "0.3%", ReserveCurrency.usdt, False),
    "NEARUSDT": (ChainId.ethereum, "uniswap-v3", "0xee8aaa362a75fbf8e0a38a50ddc27f38551e16ea", 0.656, "0.3%", ReserveCurrency.usdc, False),
    "GRTETH": (ChainId.ethereum, "uniswap-v3", "0x2e81ec0b8b4022fac83a21b2f2b4b8f5ed744d70", 0.541, "0.3%", 'ETH', False),
    "GRTUSDT": (ChainId.ethereum, "uniswap-v3", "0xb06e7ed37cfa8f0f2888355dd1913e45412798c5", 0.107, "0.3%", ReserveCurrency.usdc, False),
    "1INCHUSDT": (ChainId.ethereum, "uniswap-v3", "0x9febc984504356225405e26833608b17719c82ae", 0.415, "0.3%", ReserveCurrency.usdc, True, 1.10),
    "LITUSDT": (ChainId.binance, "pancakeswap-v2", "0xcda572e13ba703c8b72f0376f777d5ac0cd403a6", 3.2, "0.25%", ReserveCurrency.usdt, False),
    "CAKEUSDT": (ChainId.binance, "pancakeswap-v2", "0xa39af17ce4a8eb807e076805da1e2b8ea7d0755b", 1.53, "0.25%", ReserveCurrency.usdt, False),
    "BADGERBTC": (ChainId.ethereum, "uniswap-v2", "0xcd7989894bc033581532d2cd88da5db0a4b12859", 1.08, "0.3%", 'BTC', False),
    "PONDUSDT": (ChainId.ethereum, "uniswap-v2", "0xd18748b9839b0081a867a1a871d5b562b2ec9884", 0.472, "0.3%", ReserveCurrency.usdc, False),
    "ALICEUSDT": (ChainId.ethereum, "uniswap-v3", "0xde93684627d4e34d6ef96adf5bcabf10bbd8dd81", 0.141, "0.3%", ReserveCurrency.usdt),
    "BAKEUSDT": (ChainId.binance, "bakeryswap", "0x6e218ea042bef40a8baf706b00d0f0a7b4fce50a", 0.187, "0.3%", ReserveCurrency.busd, False),
    "MDXUSDT": (ChainId.binance, "erosswap", "0x223740a259e461abee12d84a9fff5da69ff071dd", 0.249, "0.3%", ReserveCurrency.busd, False),
    "MASKUSDT": (ChainId.ethereum, "uniswap-v2", "0xa40bb1c47f6dd27142a2bd7c93bfa98db9d1f5c5", 1.27, "0.3%", ReserveCurrency.usdc, False),
    "BONDUSDT": (ChainId.ethereum, "uniswap-v2", "0x6591c4bcd6d7a1eb4e537da8b78676c1576ba244", 0.234, "0.3%", ReserveCurrency.usdc, False),
    "MLNUSDT": (ChainId.ethereum, "uniswap-v3", "0x79599e9097866e3b274c00175fd92760af5e65fa", 0.31, "1%", ReserveCurrency.usdc, False),
    "FLOWUSDT": (ChainId.ethereum, "uniswap-v3", "0xd4292615a8e9edc07cba49c1a9cbb93b873a181e", 0.59152, "0.3%", ReserveCurrency.usdc, False),
    "QUICKUSDT": (ChainId.polygon, "quickswap", "0x1f1e4c845183ef6d50e9609f16f6f9cae43bc9cb", 0.177, "0.3%", ReserveCurrency.usdc, False),
    "REQUSDC": [
        (ChainId.ethereum, "uniswap-v2", "0xeb4b2b5e0eae7a0eadd0673ef8c3c830f8762f28", 0.196, "0.3%", ReserveCurrency.usdc, False),
        (ChainId.ethereum, "uniswap-v2", "0x2b9f8fe8ffc437a9008bb3097066f02b0a1c52ec", 0.185, "0.3%", ReserveCurrency.usdt, False),
    ],
    "GHSTUSDT": (ChainId.polygon, "quickswap", "0x096c5ccb33cfc5732bcd1f3195c13dbefc4c82f4", 1.05, "0.3%", ReserveCurrency.usdc, False),
    "WAXPUSDT": (ChainId.ethereum, "uniswap-v3", "0xe61cedeb1593e2d9f06af1e212842c8b5e289863", 1.04, "0.3%", ReserveCurrency.usdt, False),
    "SOLUSDC": (ChainId.binance, "babyswap", "0xae08c9d357731fd8d25681de753551be14c00405", 0.168, "0.3%", ReserveCurrency.usdc, False),
    "RADUSDT": (ChainId.ethereum, "uniswap-v2", "0x8c1c499b1796d7f3c2521ac37186b52de024e58c", 4.29, "0.3%", ReserveCurrency.usdc, False),
    "CHESSUSDT": (ChainId.binance, "pancakeswap-v2", "0x1472976e0b97f5b2fc93f1fff14e2b5c4447b64f", 0.363, "0.25%", ReserveCurrency.usdc, False),
    "ADXUSDT": (ChainId.ethereum, "uniswap-v3", "0x8de5977111c68c3fe95e63e7f7319dd5a01f77a0", 0.294, "0.3%", ReserveCurrency.usdt, False),
    "BNXUSDT": (ChainId.binance, "pancakeswap-v2", "0xb72723e36a83fb5fe1793f06b436f4720f5de4f9", 3.33, "0.25%", ReserveCurrency.busd, False),
    "ENSUSDT": (ChainId.ethereum, "uniswap-v3", "0x09aa63b7a22eefc372196aacd5b53441ed390bfb", 0.148, "0.3%", ReserveCurrency.usdt, True, 3.63),
    "HIGHUSDT": [
        (ChainId.binance, "unknown", "0xcd254486fb8f5a776d8780504b87a55b80c669dc", 0.158, "0.3%", ReserveCurrency.busd, False),
        (ChainId.binance, "pancakeswap-v2", "0xe98ac95a1db2fcaaa9c7d4ba7ecfce4877ca2bea", 0.144, "0.25%", ReserveCurrency.busd, False),
    ],
    "GMTUSDT": [
        (ChainId.polygon, "mooar", "0xb064e9b00d713496d54617c3ef6e659072b8339f", 5.57, "0.3%", ReserveCurrency.usdt, False),
        (ChainId.ethereum, "notdao", "0xaeaca8c32039a466fb32bde6f566130a1f49d21e", 2.03, "0.3%", ReserveCurrency.usdc, False),
        (ChainId.binance, "pancakeswap-v2", "0x007ec643c7cc33a70c083fc305c283dd009c8b94", 0.154, "0.25%", ReserveCurrency.usdc, False),
    ],
    "BSWUSDT": (ChainId.binance, "biswap", "0x2b30c317cedfb554ec525f85e79538d59970beb0", 0.757, "0.3%", ReserveCurrency.usdt, False),
    "GALUSDT": (ChainId.ethereum, "uniswap-v3", "0xf92f2e3fca01491baba0975264362cc38b1cab7b", 7.41, "1%", ReserveCurrency.usdc, False),
    "STGUSDT": [
        (ChainId.ethereum, "uniswap-v3", "0x8592064903ef23d34e4d5aaaed40abf6d96af186", 3.13, "1%", ReserveCurrency.usdc, False),
        (ChainId.avalanche, "trader-joe", "0x330f77bda60d8dab14d2bb4f6248251443722009", 1.63, "0.3%", ReserveCurrency.usdc, False),
        (ChainId.polygon, "susi", "0xa34ec05da1e4287fa351c74469189345990a3f0c", 1.43, "0.3%", ReserveCurrency.usdc, False),
    ],
    "HIFIUSDT": (ChainId.ethereum, "uniswap-v3", "0xc4480526e3d64f7ec7d7a6694c52d23aab658ed5", 0.303, "1%", ReserveCurrency.usdc, False),
    "PROSUSDT": (ChainId.ethereum, "uniswap-v2", "0x3155acd9f75915fcc21d34035f440da7040bd3ba", 0.279, "0.3%", ReserveCurrency.usdc, False),
    "AGIXUSDT": (ChainId.ethereum, "uniswap-v2", "0x4bb0925fa50da9b4c8936869433b48e78ccc5c13", 0.299, "0.3%", ReserveCurrency.usdt, False),
    "GNSUSDT": [
        (ChainId.arbitrum, "uniswap-v3", "0x4d2fe06fd1c4368042b926d082484d2e3cc8f3f5", 0.17764, "1%", ReserveCurrency.dai, False),
        (ChainId.polygon, "quickswap", "0x6e53cb6942e518376e9e763554db1a45ddcd25c4", 0.11057, "0.3%", ReserveCurrency.dai, False),
    ],
    "GLMUSDT": [
        (ChainId.ethereum, "uniswap-v3", "0x60257c66cee64c535ba905dd0e7a056cbb3d8658", 0.33, "1%", ReserveCurrency.usdc, False),
        (ChainId.ethereum, "uniswap-v3", "0x60a531c09dc5b6bb82ee5c6b07ad5eed6954c929", 0.216, "1%", ReserveCurrency.dai, False),
        (ChainId.ethereum, "uniswap-v3", "0x20dac1a2cf8b1bf476e144f4da439b4d8a51ea5b", 0.105, "1%", ReserveCurrency.usdt, False),
    ],
    "PROMUSDT": (ChainId.ethereum, "uniswap-v3", "0x3902428a74a08a91e2cb2fd834de75e69974fe67", 0.175, "0.3%", ReserveCurrency.usdt, False),
    "IDUSDT": (ChainId.ethereum, "uniswap-v3", "0x6ac6b053a2858bea8ad758db680198c16e523184", 4.11, "0.3%", ReserveCurrency.usdc, False),
    "ARBUSDT": [
        (ChainId.arbitrum, "uniswap-v3", "0xcda53b1f66614552f834ceef361a8d12a0b8dad8", 2.87, "0.05%", ReserveCurrency.usdc, True, 15.94),
        (ChainId.arbitrum, "uniswap-v3", "0x97bca422ec0ee4851f2110ea743c1cd0a14835a1", 0.246, "0.3%", ReserveCurrency.usdt, True, 15.94),
        (ChainId.arbitrum, "uniswap-v3", "0xb791ad21ba45c76629003b4a2f04c0d544406e37", 0.182, "0.05%", ReserveCurrency.usdt, True, 15.94),
    ],
    "PEPEUSDT": (ChainId.ethereum, "uniswap-v3", "0xcee31c846cbf003f4ceb5bbd234cba03c6e940c7", 0.153, "1%", ReserveCurrency.usdc, False),
    "WBETHETH": (ChainId.binance, "bakeryswap", "0x5fef671df9718934fda164da289374f675745d86", 2.38, "0.3%", ReserveCurrency.busd, False),
    "FDUSDUSDT": (ChainId.ethereum, "uniswap-v3", "0xcdfc3d54c8452b12285abb8c102df09ce83b8334", 0.504, "0.01%", ReserveCurrency.usdt, False),
    "CYBERUSDT": (ChainId.ethereum, "uniswap-v3", "0x05b4285b0634de1c3e72fa3ff8d54da6fdd031d8", 0.351, "1%", ReserveCurrency.usdc, False),
    "IQUSDT": (ChainId.polygon, "quickswap", "0x9f4360a2390321cb1cbff4cebeb4315d64ed3ac1", 0.16, "0.3%", ReserveCurrency.usdc, False),
    "INJUSDC": (ChainId.binance, "minereum-2", "0x38c7cef2507a2434c4f831e6f850151db7e04233", 0.191, "0.3%", ReserveCurrency.usdt, False),
    "MATICUSDC": [
        (ChainId.polygon, "uniswap-v3", "0xa374094527e1673a86de625aa59517c5de346d32", 2.88, "0.05%", ReserveCurrency.usdc, True, 53.23),
        (ChainId.polygon, "mooar", "0xc84f479bf220e38ba3bd0262049bad47aaa673ee", 1.6, "0.3%", ReserveCurrency.usdt, True, 53.23),
        (ChainId.polygon, "uniswap-v3", "0x9b08288c3be4f62bbf8d1c20ac9c5e6f9467d8b7", 1.2, "0.05%", ReserveCurrency.usdt, True, 53.23),
    ],
    "AIUSDT": (ChainId.binance, "unknown", "0x2f20634d0f0a1e9518f23943f4199dda049340af", 1.12, "0.3%", ReserveCurrency.usdt, False),
    "XAIUSDT": (ChainId.ethereum, "uniswap-v3", "0x73db82e81b7a11490278687551d3b632119b0e1f", 0.864, "0.3%", ReserveCurrency.usdc, False),
    "AXLUSDT": (ChainId.ethereum, "uniswap-v3", "0xae2a25cbdb19d0dc0dddd1d2f6b08a6e48c4a9a9", 9.43, "0.05%", ReserveCurrency.usdc, False),
    "BOMEUSDT": (ChainId.binance, "pancakeswap-v2", "0xa74e6987b104947f036cffc4a56daa22300046b6", 0.1, "0.25%", ReserveCurrency.usdt, False),
    "ETHFIUSDT": (ChainId.ethereum, "uniswap-v3", "0x80fa4c1fd0fbb9a4f071999af69531dee1016644", 0.181, "0.3%", ReserveCurrency.usdt, False),
    "ENAUSDT": [
        (ChainId.ethereum, "uniswap-v3", "0x408a625596f47314e1fd4a6cbce84c4a8695ba3f", 0.362, "0.3%", ReserveCurrency.usdc, False),
        (ChainId.ethereum, "uniswap-v3", "0xfc545f345e2afc52652cd6f76323c5ddc847bd6d", 0.107, "0.05%", ReserveCurrency.usdc, False),
    ],
    "TAOUSDT": [
        (ChainId.ethereum, "uniswap-v3", "0xf763bb342eb3d23c02ccb86312422fe0c1c17e94", 1.48, "1%", ReserveCurrency.usdc, False),
        (ChainId.ethereum, "uniswap-v3", "0x14283160198199882b5502f3316db2618baeda8c", 0.825, "0.3%", ReserveCurrency.usdc, False),
    ],
}


## Charting comparison

Here, we plot onchain and binance data side by side to see the comparison. For the purposes of the example, we use methods that aren't really used during strategy development, but are useful for visualisation purposes.

In [74]:
import datetime
from tradingstrategy.timebucket import TimeBucket

START_AT = datetime.datetime(2022,1,1)
END_AT = datetime.datetime(2024,1,1)
CANDLE_TIME_BUCKET=TimeBucket.d1

### Binance data

In [75]:
binance_candles = downloader.fetch_candlestick_data(
    symbols=["ETHUSDT"], 
    time_bucket=CANDLE_TIME_BUCKET,
    start_at=START_AT,
    end_at=END_AT,
)


### Onchain data

In [76]:
from tradingstrategy.client import Client


from pyarrow import Table
from tradingstrategy.chain import ChainId
from tradingstrategy.exchange import ExchangeUniverse
from tradingstrategy.pair import PandasPairUniverse, DEXPair

client = Client.create_jupyter_client()

# Fetch all exchange names, slugs and addresses
exchange_universe: ExchangeUniverse = client.fetch_exchange_universe()

# Fetch all trading pairs across all exchanges
pair_table: Table = client.fetch_pair_universe()
pair_universe = PandasPairUniverse(pair_table.to_pandas())

uniswap_v2 = exchange_universe.get_by_chain_and_slug(ChainId.ethereum, "uniswap-v3")

pair: DEXPair = pair_universe.get_one_pair_from_pandas_universe(
    uniswap_v2.exchange_id,
    "WETH",
    "USDC",
    pick_by_highest_vol=True)

print(f"Uniswap v2 is {uniswap_v2}")
print(f"We have trading pair {pair}")


Started Trading Strategy in Jupyter notebook environment, configuration is stored in /home/alex/.tradingstrategy
Uniswap v2 is <Exchange Uniswap v3 at 0x1f98431c8ad98523631ae4a59f267346ea31f984 on Ethereum>
We have trading pair <Pair #2711902 WETH - USDC (0xe0554a476a092703abdb3ef35c80e0d76d32939f) at exchange 3681 on ethereum>


In [77]:
import pandas as pd
from tradingstrategy.timebucket import TimeBucket

onchain_candles: pd.DataFrame = client.fetch_candles_by_pair_ids(
    {pair.pair_id},
    CANDLE_TIME_BUCKET,
    START_AT,
    END_AT,
    progress_bar_description=f"Download data for {pair.get_ticker()}"
)

### visualisations

As we can see from the side-by-side comparison, the chart data is largely the same, however the uniswap data has some spikes not present in the Binance data.

In [78]:
from tradingstrategy.charting.candle_chart import visualise_ohlcv
from tradingstrategy.utils.groupeduniverse import fix_bad_wicks

# filter MEV trades for Uniswap v3 onchain data
onchain_candles = fix_bad_wicks(onchain_candles)

onchain_figure = visualise_ohlcv(
    onchain_candles,
)

binance_figure = visualise_ohlcv(
    binance_candles,
)

In [79]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

chart_name = f"{pair.base_token_symbol} - {pair.quote_token_symbol}"
uniswap_chart_name = f"{chart_name} - Uniswap v2"
binance_chart_name = f"{chart_name} - Binance"

# Create subplot with 1 row and 2 columns
fig = make_subplots(rows=1, cols=2, subplot_titles=(uniswap_chart_name, binance_chart_name), shared_xaxes=False, shared_yaxes=False)

fig.add_trace(onchain_figure.data[1], row=1, col=1)
fig.add_trace(binance_figure.data[1], row=1, col=2)

fig.update_xaxes(rangeslider_visible=False)